# Step 2. Primera aproximación. Generacion del dataset [evidences + distractors] sin Information Retrieval.

En este script, se generan los conjuntos de datos de entrenamiento y validación, combinando los *embeddings* de los *claims* junto con los *embeddings* de las *wiki-pages* en un mismo archivo `.jsonl`.  

- El apartado **Por Fases** crea primero el dataset de evidencias y, sobre él, se insertan los distractores. Este enfoque permite un mejor manejo del proceso seguido para elegir los distractores.  
- El apartado **Completo** genera el dataset directamente con las evidencias y los distractores.  

Como resultado, se obtienen los archivos `e_train_LoRA` y `e_val_LoRA`, que contienen los *embeddings* de las evidencias de cada *claim*.  

El archivo `e_val_LoRA` incluye tanto las muestras de validación como las de prueba. Una vez generado, se divide en `val` y `test`.  

El siguiente paso es incluir *embeddings* de **detractores**, es decir, documentos que no son relevantes para la verificación del *claim*.  



In [ ]:
%pip install datasets --quiet
!python -m pip install ujson --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
import shutil
cache_dir = "/root/.cache/huggingface/datasets"  # Directorio de cache en Colab

# Elimina el directorio de cache
shutil.rmtree(cache_dir)

In [ ]:
### Librerías y código extra, necesario para la ejecución completa del cuaderno

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import word_tokenize

# Para cargar datasets del HUB de HuggingFace
from datasets import load_dataset
from datasets import Dataset, DownloadConfig

import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

import torch

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

import gc
from huggingface_hub import HfApi


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Mounted at /content/drive


# **Por fases**
1. Fase: Generacion Lora Dataset con evidencias
2. Fase: Generacion Lora Dataset con evidencias y distractores


## **1ª Fase: Combinación de los embeddings de las evidencias junto con el de los claims**

In [ ]:
from huggingface_hub import list_datasets

datasets = list_datasets(author="JORGEDC01")
print([d.id for d in datasets])

['JORGEDC01/Demo1']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data_files = {"train": "e_train.jsonl", "val": "e_val.jsonl"}
download_config = DownloadConfig(disable_tqdm=True)
dataset = load_dataset("JORGEDC01/Demo1", data_dir="Data/Embeddings_Data/bge_small_en_v1.5", data_files=data_files, download_config=download_config) # oontiene los datos de claims de FEVER
print(dataset)

#train_dataset = dataset['train']
#val_dataset = dataset['val']

df_train = pd.DataFrame(dataset['train'])
df_val = pd.DataFrame(dataset['val'])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

e_train.jsonl:   0%|          | 0.00/777M [00:00<?, ?B/s]

e_val.jsonl:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'verifiable', 'label', 'claim', 'evidence', 'claim_embedding'],
        num_rows: 145449
    })
    val: Dataset({
        features: ['id', 'verifiable', 'label', 'claim', 'evidence', 'claim_embedding'],
        num_rows: 19998
    })
})


In [ ]:
del dataset
gc.collect()

0

In [ ]:
df_train.head(2)

,id,verifiable,label,claim,evidence,claim_embedding
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[92206, 104971, Nikolaj_Coster-Waldau, 7], [9...","[-0.0048766928, 0.012141332, -0.0167379342, -0..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[174271, 187498, Roman_Atwood, 1]]","[-0.0606420673, 0.0680666119, 0.040761821, -0...."


> La siguiente celda reduce los conjuntos. *Se ha utilizado 0.1 y 0.3 en el train*. Se usa el dataset de validación completo. Una vez generado, se separa en test y validación (del mismo modo que en el paper).

In [ ]:
from sklearn.model_selection import train_test_split

train_small, _ = train_test_split(df_train, train_size=0.35, stratify=df_train["label"], random_state=42).copy()
val_small = df_val.copy()

print(f"Tamaño original train: {df_train.shape}, Tamaño reducido: {train_small.shape}")
print(f"Tamaño original val: {df_val.shape}, Tamaño reducido: {val_small.shape}")

# dataset_embeddings = load_dataset("JORGEDC01/Demo1", data_dir="wiki-pages/Embeddings_WikiPages/bge_small_en_v1.5/jsonl", streaming=True) # oontiene los datos de claims de FEVER
# iterable_ds_embeddings = dataset_embeddings["train"]

# for wikipage in iterable_ds_embeddings:
#     print(wikipage)
#     break

Tamaño original train: (145449, 6), Tamaño reducido: (50907, 6)
Tamaño original val: (19998, 6), Tamaño reducido: (19998, 6)


In [ ]:
train_small.head(2)

,id,verifiable,label,claim,evidence,claim_embedding
27207,94258,VERIFIABLE,REFUTES,Haitian Creole is a book.,"[[111682, 125488, Haitian_Creole, 0]]","[-0.0263465252, 0.0131574152, 0.0542918034, -0..."
89833,106413,VERIFIABLE,SUPPORTS,Loving tells the story of the 1967 U.S. Suprem...,"[[124915, 139246, Loving_-LRB-2016_film-RRB-, 0]]","[-0.0248791277, 0.0372825973, -0.0372912735, -..."


> **Train**

In [ ]:
import os
import pandas as pd
import ujson
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from huggingface_hub import hf_hub_download
import unicodedata

# --- Hugging Face ---
repo_id = "JORGEDC01/Demo1"
subfolder = "wiki-pages/Embeddings_WikiPages/bge_small_en_v1.5/jsonl/"
jsonl_files = [f"wiki-{i:03}.jsonl" for i in range(1, 101)]  # Desde 001 hasta 100


def normalize_text(text):
    return unicodedata.normalize("NFKC", text)


# --- Indice/Diccionario de evidencias ---
evidence_dict = {}
for _, row in train_small.iterrows():
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            # ev_id = ev[2]  # ID de la evidencia
            ev_id = normalize_text(ev[2])

            if ev_id == "None":
                continue

            if ev_id not in evidence_dict:
                evidence_dict[ev_id] = []
            evidence_dict[ev_id].append(ev)


# --- Función para procesar cada archivo JSONL ---
def process_jsonl(file_name):
    results = {}

    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=f"{subfolder}{file_name}", repo_type='dataset')

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                data = ujson.loads(line)
                # wiki_id = data["id"]
                wiki_id = normalize_text(data["id"])

                if wiki_id in evidence_dict:
                    for ev in evidence_dict[wiki_id]:
                        ev.append(data["text_embedding"])  # Se añade el embedding en el indice

                    results[wiki_id] = evidence_dict[wiki_id]

    except Exception as e:
        print(f"Error procesando {file_name}: {e}")

    return results

# --- Procesamiento en paralelo de los 100 archivos  ---
final_results = {}

with ProcessPoolExecutor(max_workers=1) as executor:  # Procesa dos archivos simultáneamente
    for partial_results in tqdm(executor.map(process_jsonl, jsonl_files), total=len(jsonl_files)):
        for key, value in partial_results.items():
            if key in final_results:
                final_results[key].extend(value)  # Evita sobrescribir, combina embeddings
            else:
                final_results[key] = value

# --- Asignar los embeddings en el DataFrame ---
for idx, row in train_small.iterrows():
    updated_evidence = []
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            # ev_id = ev[2]
            ev_id = normalize_text(ev[2])

            # Si la evidencia tiene "None" en los campos importantes, se mantiene igual
            if ev_id == "None":
                updated_evidence.append(ev)
                continue

            # if ev_id in final_results:
            #     updated_evidence.extend(final_results[ev_id])

            if ev_id in final_results:
              for stored_ev in final_results[ev_id]:
                  if stored_ev[:3] == ev[:3]:  # Verifica que sean exactamente la misma evidencia
                      updated_evidence.append(stored_ev)
                      break  # Como ya encontramos la evidencia correcta, no necesitamos seguir buscando

            else:
                updated_evidence.append(ev)  # mantiene la evidencia sin cambios si no tiene embedding

    train_small.at[idx, "evidence"] = updated_evidence  # Modifica la fila directamente

# --- Guardar en JSONL ---
output_file = "e_train_LoRA_50k.jsonl"


with open(output_file, "w", encoding="utf-8") as f:
    for _, row in train_small.iterrows():
        output_data = {}

        for field in ["verifiable", "label", "claim", "claim_embedding"]:
            if field in row:
                output_data[field] = row[field]

        output_data["evidence"] = []
        for ev in row["evidence"]:
            if isinstance(ev, list) and len(ev) > 3:
                embedding = ev[-1] if isinstance(ev[-1], list) else []
                output_data["evidence"].append(embedding)

        # Si todas las evidencias son listas vacías, deja solo una lista vacía global
        if all(isinstance(ev, list) and not ev for ev in output_data["evidence"]):
            output_data["evidence"] = []

        f.write(ujson.dumps(output_data) + "\n")

print(f"✅ Archivo guardado con formato limpio: {output_file}")

100%|██████████| 100/100 [10:20<00:00,  6.21s/it]


✅ Archivo guardado con formato limpio: e_train_LoRA_50k.jsonl


> **Development**

In [ ]:
import os
import pandas as pd
import ujson
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from huggingface_hub import hf_hub_download
import unicodedata

# --- Hugging Face ---
repo_id = "JORGEDC01/Demo1"
subfolder = "wiki-pages/Embeddings_WikiPages/bge_small_en_v1.5/jsonl/"
jsonl_files = [f"wiki-{i:03}.jsonl" for i in range(1, 101)]  # Desde 001 hasta 100


def normalize_text(text):
    return unicodedata.normalize("NFKC", text)


# --- Indice/Diccionario de evidencias ---
evidence_dict = {}
for _, row in val_small.iterrows():
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            ev_id = normalize_text(ev[2])

            if ev_id == "None":
                continue

            if ev_id not in evidence_dict:
                evidence_dict[ev_id] = []
            evidence_dict[ev_id].append(ev)


# --- Función para procesar cada archivo JSONL ---
def process_jsonl(file_name):
    results = {}

    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=f"{subfolder}{file_name}", repo_type='dataset')

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                data = ujson.loads(line)
                wiki_id = normalize_text(data["id"])

                if wiki_id in evidence_dict:
                    for ev in evidence_dict[wiki_id]:
                        ev.append(data["text_embedding"])  # Se añade el embedding en el indice

                    results[wiki_id] = evidence_dict[wiki_id]

    except Exception as e:
        print(f"Error procesando {file_name}: {e}")

    return results

# --- Procesamiento en paralelo de los 100 archivos  ---
final_results = {}

with ProcessPoolExecutor(max_workers=1) as executor:  # Procesa dos archivos simultáneamente
    for partial_results in tqdm(executor.map(process_jsonl, jsonl_files), total=len(jsonl_files)):
        for key, value in partial_results.items():
            if key in final_results:
                final_results[key].extend(value)  # Evita sobrescribir, combina embeddings
            else:
                final_results[key] = value

# --- Asignar los embeddings en el DataFrame ---
for idx, row in val_small.iterrows():
    updated_evidence = []
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            ev_id = normalize_text(ev[2])

            # Si la evidencia tiene "None" en los campos importantes, se mantiene igual
            if ev_id == "None":
                updated_evidence.append(ev)
                continue

            if ev_id in final_results:
              for stored_ev in final_results[ev_id]:
                  if stored_ev[:3] == ev[:3]:  # Verifica que sean exactamente la misma evidencia
                      updated_evidence.append(stored_ev)
                      break  # Como ya encontramos la evidencia correcta, no necesitamos seguir buscando

            else:
                updated_evidence.append(ev)  # mantiene la evidencia sin cambios si no tiene embedding

    val_small.at[idx, "evidence"] = updated_evidence  # Modifica la fila directamente

# --- Guardar en JSONL ---
output_file = "e_val_LoRA_full.jsonl"


with open(output_file, "w", encoding="utf-8") as f:
    for _, row in val_small.iterrows():
        output_data = {}

        for field in ["verifiable", "label", "claim", "claim_embedding"]:
            if field in row:
                output_data[field] = row[field]

        output_data["evidence"] = []
        for ev in row["evidence"]:
            if isinstance(ev, list) and len(ev) > 3:
                embedding = ev[-1] if isinstance(ev[-1], list) else []
                output_data["evidence"].append(embedding)

        # Si todas las evidencias son listas vacías, deja solo una lista vacía global
        if all(isinstance(ev, list) and not ev for ev in output_data["evidence"]):
            output_data["evidence"] = []

        f.write(ujson.dumps(output_data) + "\n")

print(f"✅ Archivo guardado con formato limpio: {output_file}")

100%|██████████| 100/100 [06:18<00:00,  3.78s/it]


✅ Archivo guardado con formato limpio: e_val_LoRA_full.jsonl


> División del e_val_LoRA en `val` y `test`: 10.000 muestras para el development set y 9.998 para el test set

In [ ]:
split_index = 10000

val_small = pd.read_json("e_val_LoRA_full.jsonl", lines=True)

val_split = val_small.iloc[:split_index]
test_split = val_small.iloc[split_index:]

val_output_file = "e_val_LoRA_split.jsonl"
test_output_file = "e_test_LoRA_split.jsonl"

with open(val_output_file, "w", encoding="utf-8") as f:
    for _, row in val_split.iterrows():
        f.write(ujson.dumps(row.to_dict()) + "\n")

with open(test_output_file, "w", encoding="utf-8") as f:
    for _, row in test_split.iterrows():
        f.write(ujson.dumps(row.to_dict()) + "\n")

print(f"✅ División completada. Archivos guardados como {val_output_file} y {test_output_file}.")

✅ División completada. Archivos guardados como e_val_LoRA_split.jsonl y e_test_LoRA_split.jsonl.


*e_val_LoRA_split se almacena en HuggingFace con 10k, mientra que e_test_LoRA_split tiene otras 10k. Tambien se incluye el e_val_LoRA_full con la suma de ambos, que sería 20k*

## **2ª Fase: Combinación de los embeddings de los claims y evidencias junto con los detractores**

En esta fase, existen multitud de formas para enlazar los artículos no relevantes a los claims. Entre ellas:

1. **Añadir documentos no relevantes de manera random a cada claim**


In [ ]:
import os
import pandas as pd
import ujson
import random
from tqdm import tqdm
from huggingface_hub import hf_hub_download

# --- Parámetros ---
repo_id = "JORGEDC01/Demo1"
subfolder = "wiki-pages/Embeddings_WikiPages/bge_small_en_v1.5/jsonl/"
jsonl_files = [f"wiki-{i:03}.jsonl" for i in range(1, 101)]  # Archivos de 001 a 100

distractores_por_claim = 3
file_path = "/content/e_train_LoRA_50k.jsonl"  # O "/content/e_val_lora.jsonl"
output_file = "e_train_distractors_LoRA.jsonl"  # O "e_val_distractors_LoRA.jsonl"

# --- Cargar el DataFrame con evidencias ya presentes ---
df = pd.read_json(file_path, lines=True)

max_distractors = len(df) * distractores_por_claim
distractor_count = 0

print(f"🔹 Se asignarán un máximo de {max_distractors} distractores.")

distractors = []

distractores_por_archivo = max_distractors // len(jsonl_files)
residuo = max_distractors % len(jsonl_files)

distractores_por_archivo_lista = [distractores_por_archivo] * len(jsonl_files)

if residuo > 0:
    archivos_con_residuo = random.sample(range(len(jsonl_files)), residuo)
    for idx in archivos_con_residuo:
        distractores_por_archivo_lista[idx] += 1

def obtener_distractores(file_name, distractor_per_file):
    global distractor_count

    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=f"{subfolder}{file_name}", repo_type='dataset')

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                if distractor_count >= max_distractors:
                    break
                data = ujson.loads(line)
                distractors.append(data["text_embedding"])
                distractor_count += 1

    except Exception as e:
        print(f"❌ Error procesando {file_name}: {e}")


progress_bar = tqdm(total=len(jsonl_files), desc="📥 Obteniendo distractores")

for idx, file in enumerate(jsonl_files):
    distractor_per_file = distractores_por_archivo_lista[idx]
    obtener_distractores(file, distractor_per_file)
    progress_bar.update(1)

progress_bar.close()

random.shuffle(distractors)
df["distractors"] = [[] for _ in range(len(df))]

for idx in range(len(df)):
    selected_distractors = distractors[:distractores_por_claim]
    distractors = distractors[distractores_por_claim:]
    df.at[idx, "distractors"] = selected_distractors

# --- Guardar en JSONL ---
with open(output_file, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        f.write(ujson.dumps(row.to_dict()) + "\n")

print(f"✅ Archivo guardado como {output_file}.")

# **Completo**

### Método de elección *random* distribuido en las wiki-pages

In [ ]:
from huggingface_hub import list_datasets

datasets = list_datasets(author="JORGEDC01")
print([d.id for d in datasets])

data_files = {"train": "e_train.jsonl", "val": "e_val.jsonl"}
download_config = DownloadConfig(disable_tqdm=True)
dataset = load_dataset("JORGEDC01/Demo1", data_dir="Data/Embeddings_Data/bge_small_en_v1.5", data_files=data_files, download_config=download_config) # oontiene los datos de claims de FEVER
print(dataset)

df_train = pd.DataFrame(dataset['train'])
df_val = pd.DataFrame(dataset['val'])

del dataset
gc.collect()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['JORGEDC01/Demo1']


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

e_train.jsonl:   0%|          | 0.00/777M [00:00<?, ?B/s]

e_val.jsonl:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'verifiable', 'label', 'claim', 'evidence', 'claim_embedding'],
        num_rows: 145449
    })
    val: Dataset({
        features: ['id', 'verifiable', 'label', 'claim', 'evidence', 'claim_embedding'],
        num_rows: 19998
    })
})


0

In [ ]:
from sklearn.model_selection import train_test_split

train_small, _ = train_test_split(df_train, train_size=0.35, stratify=df_train["label"], random_state=42).copy()
val_small = df_val.copy()

print(f"Tamaño original train: {df_train.shape}, Tamaño reducido: {train_small.shape}")
print(f"Tamaño original val: {df_val.shape}, Tamaño reducido: {val_small.shape}")

Tamaño original train: (145449, 6), Tamaño reducido: (50907, 6)
Tamaño original val: (19998, 6), Tamaño reducido: (19998, 6)


In [ ]:
import os
import pandas as pd
import ujson
from tqdm import tqdm
from huggingface_hub import hf_hub_download
import random

# --- Hugging Face ---
repo_id = "JORGEDC01/Demo1"
subfolder = "wiki-pages/Embeddings_WikiPages/bge_small_en_v1.5/jsonl/"
jsonl_files = [f"wiki-{i:03}.jsonl" for i in range(1, 101)]  # Desde 001 hasta 100

distractores_por_claim = 3
max_distractors = len(train_small) * distractores_por_claim
distractor_count = 0

print(f"Se asignarán un máximo de {max_distractors} distractores.")

# --- Indice/Diccionario de evidencias ---
evidence_dict = {}
for _, row in train_small.iterrows():
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            #ev_id = ev[2]  # ID de la evidencia
            ev_id = normalize_text(ev[2])

            if ev_id == "None":
                continue

            if ev_id not in evidence_dict:
                evidence_dict[ev_id] = []
            evidence_dict[ev_id].append(ev)

distractors = []

# --- Distribución de distractores entre los archivos JSONL ---
distractores_por_archivo = max_distractors // len(jsonl_files)  # Parte entera de la división
residuo = max_distractors % len(jsonl_files)  # Residuo, lo que sobra después de la división

# Crear una lista para almacenar cuántos distractores se asignan a cada archivo
distractores_por_archivo_lista = [distractores_por_archivo] * len(jsonl_files)

# Si hay residuo, distribuirlo aleatoriamente entre los archivos
if residuo > 0:
    archivos_con_residuo = random.sample(range(len(jsonl_files)), residuo)  # Escoge aleatoriamente archivos
    for idx in archivos_con_residuo:
        distractores_por_archivo_lista[idx] += 1  # Asigna 1 distractor adicional a estos archivos

# --- Función para procesar cada archivo JSONL ---
def process_jsonl(file_name, distractor_per_file):
    global distractor_count
    results = {}

    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=f"{subfolder}{file_name}", repo_type='dataset')

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                data = ujson.loads(line)
                #wiki_id = data["id"]
                wiki_id = normalize_text(data["id"])

                if wiki_id in evidence_dict:
                    for ev in evidence_dict[wiki_id]:
                        ev.append(data["text_embedding"])  # Se añade el embedding en el índice

                    results[wiki_id] = evidence_dict[wiki_id]

                else:
                    if distractor_count < max_distractors:
                        distractors.append(data["text_embedding"])
                        distractor_count += 1

    except Exception as e:
        print(f"Error procesando {file_name}: {e}")

    return results

# --- Procesar todos los archivos secuencialmente ---
final_results = {}

progress_bar = tqdm(total=len(jsonl_files), desc="Procesando archivos JSONL")

for idx, file in enumerate(jsonl_files):
    distractor_per_file = distractores_por_archivo_lista[idx]  # Obtener el número de distractores para este archivo
    partial_results = process_jsonl(file, distractor_per_file)

    for key, value in partial_results.items():
        if key in final_results:
            final_results[key].extend(value)  # Evita sobrescribir, combina embeddings
        else:
            final_results[key] = value

    progress_bar.update(1)  # Actualiza por cada archivo procesado

progress_bar.close()

random.shuffle(distractors)
train_small["distractors"] = [[] for _ in range(len(train_small))]

# --- Asigna los embeddings en el DataFrame ---
for idx, row in train_small.iterrows():
    updated_evidence = []
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            #ev_id = ev[2]
            ev_id = normalize_text(ev[2])
            # Si la evidencia tiene "None" en los campos importantes, se mantiene igual
            if ev_id == "None":
                updated_evidence.append(ev)
                continue

            # if ev_id in final_results:
            #     updated_evidence.extend(final_results[ev_id])

            if ev_id in final_results:
              for stored_ev in final_results[ev_id]:
                  if stored_ev[:3] == ev[:3]:  # Verifica que sean exactamente la misma evidencia
                      updated_evidence.append(stored_ev)
                      break  # Como ya encontramos la evidencia correcta, no necesitamos seguir buscando
            else:
                updated_evidence.append(ev)  # mantiene la evidencia sin cambios si no tiene embedding

    # --- DISTRACTORES ---
    selected_distractors = distractors[:distractores_por_claim].copy()
    distractors = distractors[distractores_por_claim:].copy()

    # Asignar los distractores a la fila
    train_small.at[idx, "distractors"] = selected_distractors
    train_small.at[idx, "evidence"] = updated_evidence


# --- Guardar en JSONL con formato limpio ---
output_file = "e_train_distractors_LoRA_50k.jsonl"

with open(output_file, "w", encoding="utf-8") as f:
    for _, row in train_small.iterrows():
        output_data = {}

        # Mantener los campos clave del DataFrame
        for field in ["verifiable", "label", "claim", "claim_embedding"]:
            if field in row:
                output_data[field] = row[field]

        # Procesar las evidencias, guardando solo los embeddings finales
        output_data["evidence"] = []
        for ev in row["evidence"]:
            if isinstance(ev, list) and len(ev) > 3:
                embedding = ev[-1] if isinstance(ev[-1], list) else []
                output_data["evidence"].append(embedding)

        # Añadir los distractores (si existen)
        output_data["distractors"] = row["distractors"]

        # Si todas las evidencias son listas vacías, deja solo una lista vacía global
        if all(isinstance(ev, list) and not ev for ev in output_data["evidence"]):
            output_data["evidence"] = []

        f.write(ujson.dumps(output_data) + "\n")

print(f"✅ Archivo guardado con formato limpio: {output_file}")


Se asignarán un máximo de 152721 distractores.


Procesando archivos JSONL: 100%|██████████| 100/100 [06:59<00:00,  4.20s/it]


✅ Archivo guardado con formato limpio: e_train_distractors_LoRA_50k.jsonl


> **Development**

In [ ]:
import os
import pandas as pd
import ujson
from tqdm import tqdm
from huggingface_hub import hf_hub_download
import random
import unicodedata

# --- Hugging Face ---
repo_id = "JORGEDC01/Demo1"
subfolder = "wiki-pages/Embeddings_WikiPages/bge_small_en_v1.5/jsonl/"
jsonl_files = [f"wiki-{i:03}.jsonl" for i in range(1, 101)]  # Desde 001 hasta 100

distractores_por_claim = 3
max_distractors = len(val_small) * distractores_por_claim
distractor_count = 0

print(f"Se asignarán un máximo de {max_distractors} distractores.")


def normalize_text(text):
    return unicodedata.normalize("NFKC", text)


# --- Indice/Diccionario de evidencias ---
evidence_dict = {}
for _, row in val_small.iterrows():
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            #ev_id = ev[2]  # ID de la evidencia
            ev_id = normalize_text(ev[2])

            if ev_id == "None":
                continue

            if ev_id not in evidence_dict:
                evidence_dict[ev_id] = []
            evidence_dict[ev_id].append(ev)

distractors = []

# --- Distribución de distractores entre los archivos JSONL ---
distractores_por_archivo = max_distractors // len(jsonl_files)  # Parte entera de la división
residuo = max_distractors % len(jsonl_files)  # Residuo, lo que sobra después de la división

# Crear una lista para almacenar cuántos distractores se asignan a cada archivo
distractores_por_archivo_lista = [distractores_por_archivo] * len(jsonl_files)

# Si hay residuo, distribuirlo aleatoriamente entre los archivos
if residuo > 0:
    archivos_con_residuo = random.sample(range(len(jsonl_files)), residuo)  # Escoge aleatoriamente archivos
    for idx in archivos_con_residuo:
        distractores_por_archivo_lista[idx] += 1  # Asigna 1 distractor adicional a estos archivos

# --- Función para procesar cada archivo JSONL ---
def process_jsonl(file_name, distractor_per_file):
    global distractor_count
    results = {}

    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=f"{subfolder}{file_name}", repo_type='dataset')

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                data = ujson.loads(line)
                #wiki_id = data["id"]
                wiki_id = normalize_text(data["id"])

                if wiki_id in evidence_dict:
                    for ev in evidence_dict[wiki_id]:
                        ev.append(data["text_embedding"])  # Se añade el embedding en el índice
                    results[wiki_id] = evidence_dict[wiki_id]

                else:
                    if distractor_count < max_distractors:
                        distractors.append(data["text_embedding"])
                        distractor_count += 1

    except Exception as e:
        print(f"Error procesando {file_name}: {e}")

    return results

# --- Procesar todos los archivos secuencialmente ---
final_results = {}

progress_bar = tqdm(total=len(jsonl_files), desc="Procesando archivos JSONL")

for idx, file in enumerate(jsonl_files):
    distractor_per_file = distractores_por_archivo_lista[idx]  # Obtener el número de distractores para este archivo
    partial_results = process_jsonl(file, distractor_per_file)

    for key, value in partial_results.items():
        if key in final_results:
            final_results[key].extend(value)  # Evita sobrescribir, combina embeddings
        else:
            final_results[key] = value

    progress_bar.update(1)  # Actualiza por cada archivo procesado

progress_bar.close()

random.shuffle(distractors)
val_small["distractors"] = [[] for _ in range(len(val_small))]

# --- Asigna los embeddings en el DataFrame ---
for idx, row in val_small.iterrows():
    updated_evidence = []
    for ev in row["evidence"]:
        if isinstance(ev, list) and len(ev) >= 3:
            #ev_id = ev[2]
            ev_id = normalize_text(ev[2])

            # Si la evidencia tiene "None" en los campos importantes, se mantiene igual
            if ev_id == "None":
                updated_evidence.append(ev)
                continue

            if ev_id in final_results:
              for stored_ev in final_results[ev_id]:
                  if stored_ev[:3] == ev[:3]:  # Verifica que sean exactamente la misma evidencia
                      updated_evidence.append(stored_ev)
                      break  # Como ya encontramos la evidencia correcta, no necesitamos seguir buscando
            else:
                updated_evidence.append(ev)  # mantiene la evidencia sin cambios si no tiene embedding

    # --- DISTRACTORES ---
    selected_distractors = distractors[:distractores_por_claim].copy()
    distractors = distractors[distractores_por_claim:].copy()

    # Asignar los distractores a la fila
    val_small.at[idx, "distractors"] = selected_distractors
    val_small.at[idx, "evidence"] = updated_evidence


# --- Guardar en JSONL con formato limpio ---
output_file = "e_val_distractors_LoRA_full.jsonl"

with open(output_file, "w", encoding="utf-8") as f:
    for _, row in val_small.iterrows():
        output_data = {}

        # Mantener los campos clave del DataFrame
        for field in ["verifiable", "label", "claim", "claim_embedding"]:
            if field in row:
                output_data[field] = row[field]

        # Procesar las evidencias, guardando solo los embeddings finales
        output_data["evidence"] = []
        for ev in row["evidence"]:
            if isinstance(ev, list) and len(ev) > 3:
                embedding = ev[-1] if isinstance(ev[-1], list) else []
                output_data["evidence"].append(embedding)

        # Añadir los distractores (si existen)
        output_data["distractors"] = row["distractors"]

        # Si todas las evidencias son listas vacías, deja solo una lista vacía global
        if all(isinstance(ev, list) and not ev for ev in output_data["evidence"]):
            output_data["evidence"] = []

        f.write(ujson.dumps(output_data) + "\n")

print(f"✅ Archivo guardado con formato limpio: {output_file}")

Se asignarán un máximo de 59994 distractores.


Procesando archivos JSONL: 100%|██████████| 100/100 [06:43<00:00,  4.04s/it]


✅ Archivo guardado con formato limpio: e_val_distractors_LoRA_full.jsonl


> **Test**

In [ ]:
split_index = 10000

val_small = pd.read_json("e_val_distractors_LoRA_full.jsonl", lines=True)

val_split = val_small.iloc[:split_index]
test_split = val_small.iloc[split_index:]

val_output_file = "e_val_distractors_LoRA_split.jsonl"
test_output_file = "e_test_distractors_LoRA_split.jsonl"

with open(val_output_file, "w", encoding="utf-8") as f:
    for _, row in val_split.iterrows():
        f.write(ujson.dumps(row.to_dict()) + "\n")

with open(test_output_file, "w", encoding="utf-8") as f:
    for _, row in test_split.iterrows():
        f.write(ujson.dumps(row.to_dict()) + "\n")

print(f"✅ División completada. Archivos guardados como {val_output_file} y {test_output_file}.")

✅ División completada. Archivos guardados como e_val_distractors_LoRA_split.jsonl y e_test_distractors_LoRA_split.jsonl.
